In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import sys
sys.path.append('d:/flagellar/code/core')
sys.path.append('/flagellar/code/core/')
sys.path.append('/kaggle/input/my-flagellar-library/')
import flg_support as fls
import flg_unet
import flg_numerics
import flg_model
import importlib
import numpy as np
import matplotlib.pyplot as plt
import time
import copy
import flg_yolo

fast_mode = False
if fls.is_submission:
    fast_mode = False

MainProcess


In [2]:
all_data = fls.load_all_train_data()
np.random.default_rng(seed=0).shuffle(all_data)

# Pick N tomograms with 1 motor and N tomograms with 0 motors as validation set
N=100
n_motors = np.array([len(d.labels) for d in all_data])
inds_zero = np.argwhere(n_motors==0)[:N,0]
inds_one = np.argwhere(n_motors==1)[:N,0]
inds_test = np.concatenate((inds_zero,inds_one))
inds_train = np.setdiff1d(np.arange(len(n_motors)), inds_test)
inds_test.shape, inds_train.shape

train_data = []
for i in inds_train:
    train_data.append(all_data[i])
test_data = []
for i in inds_test:
    test_data.append(all_data[i])
np.random.default_rng(seed=0).shuffle(test_data)
test_data = test_data
if fast_mode:
    train_data = train_data[1:30]
    test_data = test_data[2:4]
len(train_data), len(test_data)

(444, 200)

In [16]:
import pandas as pd
def get_model_for_seed(overall_seed):
    model = flg_yolo.YOLOModel()
    model.seed = overall_seed
    model.use_augs = True
    model.auto_augment = None
    rng = np.random.default_rng(seed=overall_seed)
    if rng.uniform()<0.5: model.hsv_h = 0.
    if rng.uniform()<0.5: model.hsv_s = 0.
    if rng.uniform()<0.5: model.hsv_v = 0.
    if rng.uniform()<0.5: model.translate = 0.
    if rng.uniform()<0.5: model.scale = 0.
    if rng.uniform()<0.5: model.fliplr = 0.
    if rng.uniform()<0.5: model.mosaic = 0.
    if rng.uniform()<0.5: model.mixup = 0.
    if rng.uniform()<0.5: model.erasing = 0.
    return model

res = list()
name_list = ['hsv_h', 'hsv_s', 'hsv_v', 'translate', 'scale', 'fliplr', 'mosaic', 'mixup', 'erasing']
for seed in range(20):
    m = get_model_for_seed(seed)
    d = dict()
    d['seed'] = seed
    for n in name_list:
        d[n] = 1 if getattr(m, n)>0 else 0
    res.append(d)
p=pd.DataFrame(res)
p.to_csv(fls.model_dir + 'seeds.csv')

In [5]:
for i in range(100):
    d = fls.dill_load(fls.model_dir + '/res_'+ str(i) + '.pickle')
    print(i,d[0], d[1])

0 0.8716475095785441 0.47852298417483047
1 0.8460076045627376 0.38461538461538464
2 0.8845437616387337 0.5089820359281437
3 0.8661417322834646 0.29411764705882354
4 0.8639705882352942 0.29411764705882354
5 0.6182212581344902 0.04541701073492981
6 0.8285714285714286 0.3014878621769773
7 0.7117437722419929 0.45454545454545453
8 0.7741347905282332 0.4058192955589586
9 0.8574181117533719 0.37393986121819583


FileNotFoundError: [Errno 2] No such file or directory: 'd:/flagellar/models//res_10.pickle'